# 분류 - 영화추천

## #01. 패키지 참조

In [25]:
import warnings
warnings.filterwarnings('ignore')
import os
from pandas import read_csv, DataFrame, pivot_table
from matplotlib import pyplot as plt
import seaborn as sb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
# 교차검증 패키지
from sklearn.model_selection import cross_val_score, cross_validate
import numpy as np

## #02. 데이터 가져오기
현재 디렉토리 확인

In [26]:
print(os.getcwd())

d:\jihun5\jupyter\05-데이터마이닝\02_sklearn


#### 영화 데이터 가져오기
실 분석용은 아니다. 분석후 결과 값을 맵핑하기 위한 데이터

In [27]:
origin_mv = read_csv("netflix//Netflix_Dataset_Movie.csv", encoding='utf=8')
origin_mv.head()

,Movie_ID,Year,Name
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW


In [28]:
origin_rating = read_csv("netflix//Netflix_Dataset_Rating.csv", encoding='utf-8')
origin_rating.head()

,User_ID,Rating,Movie_ID
0,712664,5,3
1,1331154,4,3
2,2632461,3,3
3,44937,5,3
4,656399,4,3


### #03. 데이터 전처리

#### 별점 데이터 재구조화

각 영화를 컬럼으로, 사용자 번호를 인덱스로 하는 피벗 테이블을 구성한다.

> 다소 시간이 오래걸림.

In [29]:
movie_users = pivot_table(origin_rating, index='Movie_ID', columns='User_ID', values='Rating')
movie_users.head()

User_ID,6,7,79,97,134,169,183,188,195,199,...,2649308,2649328,2649331,2649335,2649336,2649370,2649378,2649388,2649426,2649429
Movie_ID,,,,,,,,,,,,,,,,,,,,,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 데이터 정제

결측치는 해당 영화를 보지 않은 것으로 간주하고 0으로 대체한다.

In [30]:
movie_users.fillna(0, inplace=True)
movie_users.head()

User_ID,6,7,79,97,134,169,183,188,195,199,...,2649308,2649328,2649331,2649335,2649336,2649370,2649378,2649388,2649426,2649429
Movie_ID,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 독립변수, 종속변수 분리

x = 데이터프레임 자체 /
y = 데이터프레임의 인덱스

In [31]:
x= movie_users.copy()
y= movie_users.index
x.shape, y.shape

((1350, 143458), (1350,))

### #04. 분류 모델 구축


In [32]:
k = 5
knn = KNeighborsClassifier()
knn.fit(x, y)
y_pred = knn.predict(x)
score = accuracy_score(y, y_pred)
print("분류 정확도 : {:.2f}%".format(score))

분류 정확도 : 0.17%


In [33]:
# x.columns
# print(x.describe())
# print(x.info())

#### 교차 검증 적용

In [38]:
k_range = range(1, 30)
k_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    # score = cross_val_score(knn, x, y, cv=5).mean()
    knn.fit(x, y)
    y_pred = knn.predict(x)
    score = accuracy_score(y, y_pred)
    if np.isnan(score):
        break

    k_scores.append(score)

k_scores
    

[1.0,
 0.46074074074074073,
 0.2903703703703704,
 0.21185185185185185,
 0.1711111111111111,
 0.1474074074074074,
 0.1237037037037037,
 0.11259259259259259,
 0.1037037037037037,
 0.09555555555555556,
 0.08444444444444445,
 0.08,
 0.07703703703703704,
 0.07185185185185185,
 0.06814814814814815,
 0.06518518518518518,
 0.06222222222222222,
 0.061481481481481484,
 0.05851851851851852,
 0.056296296296296296,
 0.05407407407407407,
 0.05333333333333334,
 0.05333333333333334,
 0.05333333333333334,
 0.05185185185185185,
 0.05185185185185185,
 0.051111111111111114,
 0.05037037037037037,
 0.04666666666666667]